In [1]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup as bs

os.chdir("/home/ubuntu/real_estate_dashboard")
print(os.getcwd())

df = pd.read_csv("./0_data/legal_info_b.csv")
df

/home/ubuntu/real_estate_dashboard


,법정동코드,시도명,시군구명,읍면동명,동리명,생성일자,말소일자,법정동시군구코드,법정동읍면동코드,주소
0,1111010100,서울특별시,종로구,청운동,NaN,19880423,,11110,10100,서울특별시 종로구 청운동
1,1111010200,서울특별시,종로구,신교동,NaN,19880423,,11110,10200,서울특별시 종로구 신교동
2,1111010300,서울특별시,종로구,궁정동,NaN,19880423,,11110,10300,서울특별시 종로구 궁정동
3,1111010400,서울특별시,종로구,효자동,NaN,19880423,,11110,10400,서울특별시 종로구 효자동
4,1111010500,서울특별시,종로구,창성동,NaN,19880423,,11110,10500,서울특별시 종로구 창성동
...,...,...,...,...,...,...,...,...,...,...
20273,5013032022,제주특별자치도,서귀포시,표선면,하천리,20060701,,50130,32022,제주특별자치도 서귀포시 표선면 하천리
20274,5013032023,제주특별자치도,서귀포시,표선면,성읍리,20060701,,50130,32023,제주특별자치도 서귀포시 표선면 성읍리
20275,5013032024,제주특별자치도,서귀포시,표선면,가시리,20060701,,50130,32024,제주특별자치도 서귀포시 표선면 가시리
20276,5013032025,제주특별자치도,서귀포시,표선면,세화리,20060701,,50130,32025,제주특별자치도 서귀포시 표선면 세화리


In [2]:
# 생성한 법정코드 고윳값 찾기
LAWD_CD_list = df['법정동시군구코드'].unique()
print(len(LAWD_CD_list))
LAWD_CD_list

250


array([11110, 11140, 11170, 11200, 11215, 11230, 11260, 11290, 11305,
       11320, 11350, 11380, 11410, 11440, 11470, 11500, 11530, 11545,
       11560, 11590, 11620, 11650, 11680, 11710, 11740, 26110, 26140,
       26170, 26200, 26230, 26260, 26290, 26320, 26350, 26380, 26410,
       26440, 26470, 26500, 26530, 26710, 27110, 27140, 27170, 27200,
       27230, 27260, 27290, 27710, 28110, 28140, 28177, 28185, 28200,
       28237, 28245, 28260, 28710, 28720, 29110, 29140, 29155, 29170,
       29200, 30110, 30140, 30170, 30200, 30230, 31110, 31140, 31170,
       31200, 31710, 36110, 41111, 41113, 41115, 41117, 41131, 41133,
       41135, 41150, 41171, 41173, 41190, 41210, 41220, 41250, 41271,
       41273, 41281, 41285, 41287, 41290, 41310, 41360, 41370, 41390,
       41410, 41430, 41450, 41461, 41463, 41465, 41480, 41500, 41550,
       41570, 41590, 41610, 41630, 41650, 41670, 41800, 41820, 41830,
       42110, 42130, 42150, 42170, 42190, 42210, 42230, 42720, 42730,
       42750, 42760,

In [3]:
import time

# 실행 전 시간 기록
start_time = time.time()

# API 키 세팅
api_key = 'odgWuZqEkLq3B8i3URl%2BvXJl42Lr61W8sPd0edRlzlvelJmFSPtJ%2B%2BpdQXpU8lhVdGN1yvYOHsuRJNH8G3m6aQ%3D%3D'

# 컬럼
columns = ['갱신요구권사용',
'건축년도',
'계약구분',
'계약기간',
'년',
'법정동',
'보증금액',
'아파트',
'월',
'월세금액',
'일',
'전용면적',
'종전계약보증금',
'종전계약월세',
'지번',
'지역코드',
'층']

# 서비스 url
url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent"

# 에러 추출용
error = []

# 데이터 저장용
total = pd.DataFrame()

for i in range(len(LAWD_CD_list)):
    
    # 데이터를 담는 리스트
    df_total = []
    
    # 추출하는 연월
    year_month_key = "202101"

    # 요청변수 파라미터 설정
    params = "?" + \
             "serviceKey=" + api_key + "&" + \
             "LAWD_CD=" + str(LAWD_CD_list[i]) + "&" + \
             "DEAL_YMD=" + year_month_key
    
    res = requests.get(url + params)
    soup = bs(res.text, 'xml')
    items = soup.find_all('item')
    
    if(len(items) > 0):
        # 수집된 데이터를 데이터 프레임 형식으로 변환
        for j in range(len(items)):
            df_raw = []
            for column in columns:
                # 컬럼 데이터가 없는 경우 처리
                try:
                    df_raw.append(items[j].find(column).text)
                except:
                    print(LAWD_CD_list[i], j , column) # 에러 컬럼 파악 여부
                    df_raw.append('없음')
            # 데이터 행 하나를 더해줍니다        
            df_total.append(df_raw)
            
    else:
        # 해당 지역에 거래 데이터가 없는 경우
        error.append(LAWD_CD_list[i])
        continue
        
    df = pd.DataFrame(df_total)
    df.columns = columns
    
    total = pd.concat([total,df])
    total.columns = columns
    
    print(f"{LAWD_CD_list[i]} 데이터 추출 완료")
        
        
        
        
df = pd.read_csv("./0_data/legal_info_b.csv")      

for i in range(len(error)):
     print(f'{error[i]} 거래 데이터가 없습니다')
    
# 실행 후 시간 기록
end_time = time.time()    
    
# 실행 시간 계산
execution_time = end_time - start_time
print(f"실행 시간: {execution_time} 초")   
    
total.reset_index(drop=True,inplace=True)
total

11110 데이터 추출 완료
11140 데이터 추출 완료
11170 데이터 추출 완료
11200 데이터 추출 완료
11215 데이터 추출 완료
11230 데이터 추출 완료
11260 데이터 추출 완료
11290 데이터 추출 완료
11305 데이터 추출 완료
11320 데이터 추출 완료
11350 데이터 추출 완료
11380 데이터 추출 완료
11410 데이터 추출 완료
11440 데이터 추출 완료
11470 데이터 추출 완료
11500 데이터 추출 완료
11530 데이터 추출 완료
11545 데이터 추출 완료
11560 데이터 추출 완료
11590 데이터 추출 완료
11620 데이터 추출 완료
11650 969 지번
11650 982 지번
11650 데이터 추출 완료
11680 데이터 추출 완료
11710 데이터 추출 완료
11740 데이터 추출 완료
26110 데이터 추출 완료
26140 데이터 추출 완료
26170 데이터 추출 완료
26200 데이터 추출 완료
26230 데이터 추출 완료
26260 데이터 추출 완료
26290 데이터 추출 완료
26320 데이터 추출 완료
26350 데이터 추출 완료
26380 데이터 추출 완료
26410 데이터 추출 완료
26440 데이터 추출 완료
26470 데이터 추출 완료
26500 데이터 추출 완료
26530 데이터 추출 완료
26710 데이터 추출 완료
27110 데이터 추출 완료
27140 데이터 추출 완료
27170 데이터 추출 완료
27200 데이터 추출 완료
27230 데이터 추출 완료
27260 데이터 추출 완료
27290 데이터 추출 완료
27710 데이터 추출 완료
28110 데이터 추출 완료
28140 데이터 추출 완료
28177 데이터 추출 완료
28185 데이터 추출 완료
28200 데이터 추출 완료
28237 데이터 추출 완료
28245 데이터 추출 완료
28260 데이터 추출 완료
28710 데이터 추출 완료
29110 데이터 추출 완료
29140 데이터 추출 완료
29155 데이터 추출 완

,갱신요구권사용,건축년도,계약구분,계약기간,년,법정동,보증금액,아파트,월,월세금액,일,전용면적,종전계약보증금,종전계약월세,지번,지역코드,층
0,,2008,,,2021,사직동,"20,000",광화문스페이스본(106동),1,280,5,150.4,,,9-1,11110,5
1,,2008,,,2021,사직동,"3,000",광화문스페이스본(101동~105동),1,300,7,94.51,,,9,11110,9
2,,2008,,,2021,사직동,"50,000",광화문스페이스본(101동~105동),1,150,9,108.55,,,9,11110,1
3,,2008,,,2021,사직동,"80,000",광화문스페이스본(101동~105동),1,0,9,70.8,,,9,11110,12
4,,2008,,,2021,사직동,"120,000",광화문스페이스본(101동~105동),1,0,12,158.99,,,9,11110,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74578,,2010,,,2021,대정읍 하모리,"1,439",서귀포대정휴먼시아,1,12,18,46.8,,,1450,50130,5
74579,,2019,,,2021,대정읍 하모리,"1,000",비슬휴림,1,67,22,65.8886,,,1089-22,50130,5
74580,,1996,,,2021,남원읍 남원리,"7,000",에덴,1,0,20,40,,,1323-2,50130,3
74581,,2020,,,2021,성산읍 고성리,699,서귀포성산엘에이치아파트,1,12,2,26.44,,,1141-1,50130,7


In [4]:
directory = '0_data/apt_rent'

if not os.path.exists(directory):
    os.makedirs(directory)

file_name = f"{directory}/apt_rent_{str(year_month_key)}.csv"

total.to_csv(file_name, index = False)